In [1]:
#Imports
import logging
import os, random, time, sys
import numpy as np
import tensorflow as tf
import tensorflow.contrib.layers as layers
import ipdb
import keras
import pandas as pd

sys.path.append('../../../')
import feedforward_robust as ffr

sys.path.append('../../../utils/')
from utils.mnist_corruption import *
from utils.utils_models import *
from utils.utils_analysis import *
from utils.utils_feedforward import *

%load_ext autoreload
%autoreload 2

#Read the counter
ctr_file = "../../counter.txt"
f = open(ctr_file, 'r')
counter = f.readline()
f.close()

counter = 1 + int(counter)
f = open(ctr_file,'w')
f.write('{}'.format(counter))
f.close()
logfile = "../../logs/results_" + str(counter) + ".log"

logger = logging.getLogger("robustness")
logger.setLevel(logging.DEBUG)
fh = logging.FileHandler(logfile)
fh.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
logger.addHandler(fh)
logger.addHandler(logging.StreamHandler())

Using TensorFlow backend.
W0611 14:30:40.103560 4439213504 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/cleverhans/utils_tf.py:341: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.



# Get Fashion MNIST Data

In [2]:
fashion_mnist = keras.datasets.fashion_mnist

In [3]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train = x_train/255
x_test = x_test/255

In [4]:
#One hot the labels
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)                                                                                         
y_test = keras.utils.to_categorical(y_test, num_classes)

In [5]:
x_train_master, y_train_master = x_train[:30000], y_train[:30000]
x_train_slave, y_train_slave = x_train[30000:], y_train[30000:]

In [6]:
#Flatten everything
x_train_master_flat, input_shape = flatten_mnist(x_train_master) 
x_train_slave_flat, _ = flatten_mnist(x_train_slave)
x_test_flat, _  = flatten_mnist(x_test)

# Train Master

In [7]:
#Configurations
eps_train = 0.1                                                                                                                            
eps_test = 0.1                                                                                                                             
tensorboard_dir = "../tb/"                                                                                                                
weights_dir = "../weights/"                                                                                                               
load_weights = False                                                                                                              
load_counter = 234                                                                                                            
sigma = tf.nn.relu                                                                                                                         
epochs, reg, lr = 1, 0.00, 1e-3    
#epochs, reg, lr = 30, 0.00, 15e-4                                                                                                          
pgd_eta, pgd_num_iter = 1e-2, 50

In [8]:
#Setup - Dataset stuff
tf.reset_default_graph()
sess = tf.Session()
hidden_sizes = [64, 64, 32]
dataset = ((x_train_master_flat, y_train_master), (x_test_flat, y_test))

scope_name = "teacher_student_fashion"
if not load_weights:
    with tf.variable_scope(scope_name, reuse = False) as scope:

        logdir = tensorboard_dir + str(counter)

        #Create model
        writer = tf.summary.FileWriter(logdir)
        model = ffr.RobustMLP(input_shape, hidden_sizes, num_classes, writer = writer, scope = scope_name, logger = logger, sigma = sigma)
        sess.run(tf.global_variables_initializer())
        print("Created model successfully. Now going to train")
    
        #Train model
        model.fit(sess, x_train_master_flat, y_train_master, training_epochs = epochs, reg_op = reg , lr = lr)
        
        """
        #Save weights
        weights = tf.trainable_variables()
        #weights = model.get_weights()[0] + model.get_weights()[1]
        saver = tf.train.Saver(weights)
        weights_path = saver.save(sess, weights_dir + "model_" + str(counter) + ".ckpt")
        print("Saved model at %s"%weights_path)
        """
        
        #Test model - regular, fgsm adv, pgd adv
        loss_reg, acc_reg = model.evaluate(sess, x_test_flat, y_test)
        print("----Regular test accuracy and loss ----")
        print((loss_reg, acc_reg))
        
        loss_fgsm, acc_fgsm = model.adv_evaluate(sess, x_test_flat, y_test, eps_test, pgd = False)
        print("----FGSM test accuracy and loss ----")
        print((loss_fgsm, acc_fgsm))
        
        loss_pgd, acc_pgd = model.adv_evaluate(sess, x_test_flat, y_test, eps_test, pgd = True, eta=pgd_eta, num_iter = pgd_num_iter)
        print("----PGD test accuracy and loss ----")
        print((loss_pgd , acc_pgd))

Initialized instance variables of the robust model class
I0611 14:30:41.150832 4439213504 feedforward_robust.py:32] Initialized instance variables of the robust model class
W0611 14:30:41.153137 4439213504 deprecation_wrapper.py:119] From ../../../feedforward_robust.py:36: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

Created placeholders for x and y
I0611 14:30:41.161875 4439213504 feedforward_robust.py:40] Created placeholders for x and y
W0611 14:30:41.163236 4439213504 deprecation_wrapper.py:119] From ../../../utils/utils_feedforward.py:34: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0611 14:30:41.163862 4439213504 deprecation_wrapper.py:119] From ../../../utils/utils_feedforward.py:34: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W0611 14:30:41.164582 4439213504 deprecation_wrapper.py:119] From ../../../utils/utils_feedforward.py:36: The name tf.get_variable

Created model successfully. Now going to train


W0611 14:30:41.814190 4439213504 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow/python/util/tf_should_use.py:193: initialize_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Use `tf.variables_initializer` instead.
Epoch: 0001    cost: 0.609549703 
I0611 14:30:43.120464 4439213504 feedforward_robust.py:716] Epoch: 0001    cost: 0.609549703 
Accuracy on batch: 0.812500
I0611 14:30:43.121809 4439213504 feedforward_robust.py:717] Accuracy on batch: 0.812500
Optimization Finished!
I0611 14:30:43.123045 4439213504 feedforward_robust.py:718] Optimization Finished!
Final Train Loss 0.434829
I0611 14:30:43.305584 4439213504 feedforward_robust.py:726] Final Train Loss 0.434829
Final Train Accuracy 0.843000:
I0611 14:30:43.308176 4439213504 feedforward_robust.py:727] Final Train Accuracy 0.843000:
Model was trained on benign data
I0611 14:30:43.310034 4439213504 feedforward_robust.py:745] Mo

----Regular test accuracy and loss ----
(0.48024386, 0.8267)


Model is being evaluated on FGSM data
I0611 14:30:43.635019 4439213504 feedforward_robust.py:649] Model is being evaluated on FGSM data
Model is being evaluated on PGD points generated using 0.010000 learning rate and 50 iterations
I0611 14:30:43.694988 4439213504 feedforward_robust.py:651] Model is being evaluated on PGD points generated using 0.010000 learning rate and 50 iterations


----FGSM test accuracy and loss ----
(5.2600427, 0.037)
iteration: 0
loss 0.702678
iteration: 1
loss 0.992560
iteration: 2
loss 1.353859
iteration: 3
loss 1.775042
iteration: 4
loss 2.236981
iteration: 5
loss 2.719240
iteration: 6
loss 3.205202
iteration: 7
loss 3.681864
iteration: 8
loss 4.140436
iteration: 9
loss 4.575137
iteration: 10
loss 4.981952
iteration: 11
loss 5.357927
iteration: 12
loss 5.700738
iteration: 13
loss 6.009244
iteration: 14
loss 6.281857
iteration: 15
loss 6.518266
iteration: 16
loss 6.717598
iteration: 17
loss 6.878910
iteration: 18
loss 7.001528
iteration: 19
loss 7.084291
iteration: 20
loss 7.143650
iteration: 21
loss 7.196340
iteration: 22
loss 7.242820
iteration: 23
loss 7.283857
iteration: 24
loss 7.319435
iteration: 25
loss 7.351280
iteration: 26
loss 7.378484
iteration: 27
loss 7.402395
iteration: 28
loss 7.422898
iteration: 29
loss 7.441172
iteration: 30
loss 7.456722
iteration: 31
loss 7.470504
iteration: 32
loss 7.482231
iteration: 33
loss 7.492923
it

This is to confirm that attack does not violate constraints
I0611 14:30:48.852742 4439213504 feedforward_robust.py:496] This is to confirm that attack does not violate constraints
Should be no more than eps
I0611 14:30:48.854115 4439213504 feedforward_robust.py:497] Should be no more than eps


loss 7.565024
iteration: 49
loss 7.567389


0.10000005352730845
I0611 14:30:48.918920 4439213504 feedforward_robust.py:498] 0.10000005352730845


----PGD test accuracy and loss ----
(7.567389, 0.0141)


## Generate data for slave

In [9]:
with tf.variable_scope(scope_name, reuse = False) as scope:
        loss_reg, acc_reg = model.evaluate(sess, x_test_flat, y_test)
        print("----Regular test accuracy and loss ----")
        print((loss_reg, acc_reg))
        
        z_train_slave = model.get_prediction(sess, x_train_slave_flat)
        z_test_slave = model.get_prediction(sess, x_test_flat)

Model was evaluated on benign data
I0611 14:30:49.023163 4439213504 feedforward_robust.py:642] Model was evaluated on benign data


----Regular test accuracy and loss ----
(0.48024386, 0.8267)


In [10]:
z_train_slave[0]

array([-0.31469756,  1.8906322 , -2.1341326 ,  6.425183  , -0.7633205 ,
       -5.4411635 , -2.3879952 , -8.553714  , -3.4101493 , -8.622425  ],
      dtype=float32)

# Train slave

In [20]:
#Setup - Dataset stuff
def slave_training():
    epochs = 20
    lr = 15e-4
    tf.reset_default_graph()
    sess = tf.Session()
    hidden_sizes = [64, 64, 32]
    dataset = ((x_train_slave_flat, z_train_slave), (x_test_flat, y_test))

    scope_name = "teacher_student_fashion"
    if not load_weights:
        with tf.variable_scope(scope_name, reuse = tf.AUTO_REUSE) as scope:

            logdir = tensorboard_dir + str(counter)

            #Create model
            writer = tf.summary.FileWriter(logdir)
            model = ffr.RobustMLP(input_shape, hidden_sizes, num_classes, writer = writer, scope = scope_name, logger = logger, sigma = sigma, classification = False)
            sess.run(tf.global_variables_initializer())
            print("Created model successfully. Now going to train")

            #Train model
            model.fit(sess, x_train_slave_flat, z_train_slave, training_epochs = epochs, reg_op = reg , lr = lr)

            """
            #Save weights
            weights = tf.trainable_variables()
            #weights = model.get_weights()[0] + model.get_weights()[1]
            saver = tf.train.Saver(weights)
            weights_path = saver.save(sess, weights_dir + "model_" + str(counter) + ".ckpt")
            print("Saved model at %s"%weights_path)
            """
            loss_real_train, acc_train = model.evaluate(sess, x_train_slave_flat, z_train_slave)

            #Test model - regular, fgsm adv, pgd adv
                        
            loss_class_reg, acc_reg = model.evaluate(sess, x_test_flat, y_test)
            print("----Regular test loss and accuracy ----")
            print((loss_class_reg, acc_reg))
            
            loss_real_reg, acc_real_reg = model.evaluate(sess, x_test_flat, z_test_slave)
            print("----Real test loss and accuracy comparing to teacher ----")
            print((loss_real_reg, acc_real_reg))

            loss_fgsm, acc_fgsm = model.adv_evaluate(sess, x_test_flat, y_test, eps_test, pgd = False)
            print("----FGSM test loss and accuracy ----")
            print((loss_fgsm, acc_fgsm))

            loss_pgd, acc_pgd = model.adv_evaluate(sess, x_test_flat, y_test, eps_test, pgd = True, eta=pgd_eta, num_iter = pgd_num_iter)
            print("----PGD test loss and accuracy ----")
            print((loss_pgd , acc_pgd))
            
            slave_train_confidences = model.get_prediction(sess, x_train_slave_flat)
            slave_test_confidences = model.get_prediction(sess, x_test_flat)
            
            return loss_real_train, acc_train, loss_real_reg, acc_real_reg, acc_reg, slave_train_confidences, slave_test_confidences
        
        

In [21]:
tup = slave_training()

Initialized instance variables of the robust model class
I0611 14:42:09.541872 4439213504 feedforward_robust.py:32] Initialized instance variables of the robust model class
Created placeholders for x and y
I0611 14:42:09.549396 4439213504 feedforward_robust.py:40] Created placeholders for x and y
Created layers and tensor for logits
I0611 14:42:09.649940 4439213504 feedforward_robust.py:44] Created layers and tensor for logits
Added accuracy computation to the graph
I0611 14:42:09.656579 4439213504 feedforward_robust.py:48] Added accuracy computation to the graph
Added MSE loss computation to the graph
I0611 14:42:09.663799 4439213504 feedforward_robust.py:60] Added MSE loss computation to the graph
Model graph was created
I0611 14:42:09.665156 4439213504 feedforward_robust.py:62] Model graph was created


Created model successfully. Now going to train


Epoch: 0001    cost: 2.237462381 
I0611 14:42:11.227638 4439213504 feedforward_robust.py:716] Epoch: 0001    cost: 2.237462381 
Accuracy on batch: 0.906250
I0611 14:42:11.228815 4439213504 feedforward_robust.py:717] Accuracy on batch: 0.906250
Epoch: 0002    cost: 0.260243641 
I0611 14:42:12.117221 4439213504 feedforward_robust.py:716] Epoch: 0002    cost: 0.260243641 
Accuracy on batch: 0.906250
I0611 14:42:12.119945 4439213504 feedforward_robust.py:717] Accuracy on batch: 0.906250
Epoch: 0003    cost: 0.196668937 
I0611 14:42:13.158215 4439213504 feedforward_robust.py:716] Epoch: 0003    cost: 0.196668937 
Accuracy on batch: 0.937500
I0611 14:42:13.159495 4439213504 feedforward_robust.py:717] Accuracy on batch: 0.937500
Epoch: 0004    cost: 0.166499397 
I0611 14:42:14.130656 4439213504 feedforward_robust.py:716] Epoch: 0004    cost: 0.166499397 
Accuracy on batch: 0.937500
I0611 14:42:14.131684 4439213504 feedforward_robust.py:717] Accuracy on batch: 0.937500
Epoch: 0005    cost: 0.1

----Regular test loss and accuracy ----
(65.91971, 0.8262)
----Real test loss and accuracy comparing to teacher ----
(0.09702799, 0.9754)


Model is being evaluated on FGSM data
I0611 14:42:32.918411 4439213504 feedforward_robust.py:649] Model is being evaluated on FGSM data
Model is being evaluated on PGD points generated using 0.010000 learning rate and 50 iterations
I0611 14:42:32.936987 4439213504 feedforward_robust.py:651] Model is being evaluated on PGD points generated using 0.010000 learning rate and 50 iterations


----FGSM test loss and accuracy ----
(202.65367, 0.6256)
iteration: 0
loss 0.737341
iteration: 1
loss 1.070574
iteration: 2
loss 1.491013
iteration: 3
loss 1.980300
iteration: 4
loss 2.514758
iteration: 5
loss 3.068840
iteration: 6
loss 3.622521
iteration: 7
loss 4.161366
iteration: 8
loss 4.676623
iteration: 9
loss 5.162650
iteration: 10
loss 5.615674
iteration: 11
loss 6.033039
iteration: 12
loss 6.412905
iteration: 13
loss 6.754102
iteration: 14
loss 7.055805
iteration: 15
loss 7.316837
iteration: 16
loss 7.537161
iteration: 17
loss 7.715266
iteration: 18
loss 7.850503
iteration: 19
loss 7.942114
iteration: 20
loss 8.008368
iteration: 21
loss 8.067415
iteration: 22
loss 8.119392
iteration: 23
loss 8.165595
iteration: 24
loss 8.206116
iteration: 25
loss 8.241647
iteration: 26
loss 8.272553
iteration: 27
loss 8.299675
iteration: 28
loss 8.323347
iteration: 29
loss 8.344050
iteration: 30
loss 8.362108
iteration: 31
loss 8.378006
iteration: 32
loss 8.392079
iteration: 33
loss 8.404270
i

This is to confirm that attack does not violate constraints
I0611 14:42:37.870707 4439213504 feedforward_robust.py:496] This is to confirm that attack does not violate constraints
Should be no more than eps
I0611 14:42:37.872554 4439213504 feedforward_robust.py:497] Should be no more than eps


loss 8.490906


0.10000005352730845
I0611 14:42:37.961200 4439213504 feedforward_robust.py:498] 0.10000005352730845


----PGD test loss and accuracy ----
(111.69142, 0.0108)


In [13]:
a

NameError: name 'a' is not defined

In [ ]:
list(tup)

In [ ]:
df = pd.DataFrame()
df['mse on z_train'] = []
df['acc on z_train'] = []
df['mse on z_test'] = []
df['acc on z_test'] = []
df['acc on y_test'] = []

In [ ]:
train_confidences = []
test_confidences = []
for i in range(3):
    loss_real_train, acc_train, loss_real_reg, acc_real_reg, acc_reg, slave_train_confidences, slave_test_confidences = slave_training()
    df.loc[i] = [loss_real_train, acc_train, loss_real_reg, acc_real_reg, acc_reg]
    train_confidences.append(slave_train_confidences)
    test_confidences.append(slave_test_confidences)

In [ ]:
df

In [ ]:
#df.to_excel("ts_fashion_results.xlsx")  